In [ ]:
from time import time
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import Flatten
from keras.utils import np_utils

tiempo_inicial = time() 
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()
X_train = X_train.reshape(60000, 28, 28, 1)
X_test = X_test.reshape(10000, 28, 28, 1)
Y_train = np_utils.to_categorical(Y_train, 10)
Y_test = np_utils.to_categorical(Y_test, 10)

model = Sequential([Convolution2D(filters=32,
                                  kernel_size=(3, 3),
                                  input_shape=(28, 28, 1)), # first conv layer
                    Activation('relu'),
                    Convolution2D(filters=32,
                                  kernel_size=(3, 3)), # second conv layer
                    Activation('relu'),
                    MaxPooling2D(pool_size=(2, 2)), # max pooling layer
                    Flatten(), # flatten the output tensor
                    Dense(64), # fully-connected hidden layer
                    Activation('relu'),
                    Dense(10), # output layer
                    Activation('softmax')])
print(model.summary())
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adadelta')
model.fit(X_train, Y_train, batch_size=10, epochs=1, validation_split=0.1, verbose=1)
score = model.evaluate(X_test, Y_test, verbose=1)
print('Test accuracy:', score[1])

tiempo_final = time() 
tiempo_ejecucion = tiempo_final - tiempo_inicial
print ('El tiempo de ejecucion fue:',tiempo_ejecucion)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 activation_4 (Activation)   (None, 26, 26, 32)        0         
                                                                 
 conv2d_3 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 activation_5 (Activation)   (None, 24, 24, 32)        0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 12, 12, 32)       0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 4608)              0         
                                                      

Tiempo de ejecucion con CPU: 844.8924615383148

Tiempo de ejecucion con GPU: 47.8739800453186

Comparacion de resultados:
Secuencial:

loss: 0.2417 - acc: 0.9336

Convolucional:

loss: 8.0932e-04 - acc: 0.9997 - val_loss: 0.0758 - val_acc: 0.9897



In [ ]:
!pip install optuna
import optuna
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

def objective(trial):
    model = Sequential([Convolution2D(filters=32,
                                  kernel_size=(3, 3),
                                  input_shape=(28, 28, 1)), # first conv layer
                    Activation('relu'),
                    Convolution2D(filters=32,
                                  kernel_size=(3, 3)), # second conv layer
                    Activation('relu'),
                    MaxPooling2D(pool_size=(2, 2)), # max pooling layer
                    Flatten(), # flatten the output tensor
                    Dense(64), # fully-connected hidden layer
                    Activation('relu'),
                    Dense(10), # output layer
                    Activation('softmax')])
    # Definir los hiperparámetros para optimizar con Optuna
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    momentum = trial.suggest_float("momentum", 0.0, 1.0)
    batch_size = trial.suggest_categorical("batch_size", [32, 64, 128])

    # Compilar el modelo con los hiperparámetros elegidos
    #optimizer = tf.keras.optimizers.SGD(learning_rate=lr, momentum=momentum)
    #model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adadelta')
    # Entrenar el modelo con los hiperparámetros elegidos
    history = model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=2, batch_size=batch_size)

    # Devolver el valor de pérdida en la última época como objetivo a minimizar por Optuna
    return history.history['val_loss'][-1]

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=2)

# Obtener los mejores hiperparámetros encontrados
best_params = study.best_params
print("Mejores hiperparámetros:", best_params)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[I 2023-04-12 20:54:54,449] A new study created in memory with name: no-name-dd8034ac-35eb-4c4c-b73d-f72a957327a0


Epoch 1/2
1875/1875 [==============================] - 13s 5ms/step - loss: 11.6885 - accuracy: 0.2655 - val_loss: 4.0615 - val_accuracy: 0.4590
Epoch 2/2
1875/1875 [==============================] - 8s 4ms/step - loss: 2.6809 - accuracy: 0.5750 - val_loss: 1.7837 - val_accuracy: 0.6736


[I 2023-04-12 20:55:36,720] Trial 0 finished with value: 1.7836552858352661 and parameters: {'lr': 6.304951547212558e-05, 'momentum': 0.48457032912923004, 'batch_size': 32}. Best is trial 0 with value: 1.7836552858352661.


Epoch 1/2
1875/1875 [==============================] - 9s 4ms/step - loss: 8.7473 - accuracy: 0.2852 - val_loss: 3.3109 - val_accuracy: 0.4924
Epoch 2/2
1875/1875 [==============================] - 9s 5ms/step - loss: 2.4258 - accuracy: 0.5724 - val_loss: 1.8217 - val_accuracy: 0.6459


[I 2023-04-12 20:55:54,881] Trial 1 finished with value: 1.8217082023620605 and parameters: {'lr': 0.07021013949887808, 'momentum': 0.5099081778916084, 'batch_size': 32}. Best is trial 0 with value: 1.7836552858352661.


Mejores hiperparámetros: {'lr': 6.304951547212558e-05, 'momentum': 0.48457032912923004, 'batch_size': 32}
